l'exécution a été réalisée sur Google Collab en package pro. En raison du traitement des images en arrays et dataframe, le ram utilisé est importante et afin de réduire les délais d'execution, l'ensemble des fichiers CSV était déjà créés et stockés dans le drive.

L'execution de ce script doit se faire une fois ces fichiers CSV créés et présents sur le drive.

Approche non supervisée utilisation l'algorythme de l'isolation forest sur les features (pixel) de l'image. La recherche des paramètre optimum via Grid search n'a pas permi d'exploiter cette approche.

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
##Création du dataframe total contenant toutes les données pour toutes les images
df_total=pd.read_csv('drive/MyDrive/Projet Son/df_total.csv',header=0,index_col=0)

In [ ]:
#La realisation d'un pca et/ou l'execution de modèles sur des données de si grande dimension étant impossible même avec des ressources Ram très importantes, nous le lançons sur une partie seulement du jeu de donnée en partant du principe qu'on pourra le generaliser à l'ensemble du modèle
df_total_topca = df_total.sample(frac=0.4)


In [ ]:
df_total_topca['Target_ano'].value_counts()

0    10173
1     2222
Name: Target_ano, dtype: int64

In [ ]:
#Preparation des données 
target = df_total_topca['Target_ano']
data = df_total_topca.drop(['Machine','ID','file','Target_ano'],axis=1)

In [ ]:
#Réalisation d'un PCA pour ne conserver que les features (pixel) qui génèrent de la variance.
#L'idée était de partir sur l'hypothèse qu'une majorité de pixels des spectogrammes ne sont pas utiles à l'analyse.
from sklearn.decomposition import PCA
pca = PCA(n_components = 0.9)
pca.fit(data)
print("Nombre de composantes retenues :", pca.n_components_)

Nombre de composantes retenues : 1106


In [10]:
Data_pca=pca.transform(data)

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(Data_pca,target,test_size = .2)

In [18]:
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn import model_selection

model = IsolationForest(random_state=47)

param_grid = {'n_estimators': [1000, 1500], 
              'max_samples': [10], 
              'contamination': ['auto', 0.0001, 0.0002], 
              'max_features': [10, 15], 
              'bootstrap': [True], 
              'n_jobs': [-1]}

grid_search = model_selection.GridSearchCV(model, 
                                           param_grid,
                                           scoring="neg_mean_squared_error", 
                                           refit=True,
                                           cv=10, 
                                           return_train_score=True)
grid_search.fit(X_train, y_train)

best_model = grid_search.fit(X_train, y_train)
print('Optimum parameters', best_model.best_params_)


Optimum parameters {'bootstrap': True, 'contamination': 0.0001, 'max_features': 10, 'max_samples': 10, 'n_estimators': 1000, 'n_jobs': -1}


In [24]:
Y_score=best_model.decision_function(X_train)
Y_pred=best_model.predict(X_train)
pd.crosstab(Y_pred,y_train)

# Les résultats obtenus sont mauvais. aucune des anomalies (1) n'a été identifié par le modèle.



Target_ano,0,1
row_0,,
-1,1,0
1,8145,1770
